<a href="https://colab.research.google.com/github/RICHAR-SL/Proyectos/blob/main/Pinta%20mi%20emoci%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install fer

In [1]:
# ==========================================================
# 🔹 Librerías de sistema y manejo de archivos
# ==========================================================
import os
import zipfile
import glob

# ==========================================================
# 🔹 Manejo numérico y científico
# ==========================================================
import numpy as np
import pandas as pd

# ==========================================================
# 🔹 Procesamiento de imágenes
# ==========================================================
import cv2
from PIL import Image

# ==========================================================
# 🔹 Visualización
# ==========================================================
import matplotlib.pyplot as plt
import seaborn as sns

# ==========================================================
# 🔹 Preprocesamiento y datasets
# ==========================================================
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# ==========================================================
# 🔹 Deep Learning (TensorFlow / Keras)
# ==========================================================
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# ==========================================================
# 🔹 Interfaz web
# ==========================================================
import gradio as gr


In [18]:
# Función para descomprimir
def descomprimir(zip_path, extract_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
    print(f"✅ Extraído en: {extract_path}")

# Rutas a tus zips
fer_zip = "/content/FER2013.zip"
affectnet_zip = "/content/archive.zip"

# Carpetas de salida
fer_out = "/content/dataset/FER2013"
affectnet_out = "/content/dataset/AffectNet"

# Descomprimir
descomprimir(fer_zip, fer_out)
descomprimir(affectnet_zip, affectnet_out)


✅ Extraído en: /content/dataset/FER2013
✅ Extraído en: /content/dataset/AffectNet


In [38]:
!pip uninstall -y keras


Found existing installation: keras 3.10.0
Uninstalling keras-3.10.0:
  Successfully uninstalled keras-3.10.0


In [19]:
import tensorflow as tf

img_size = (48, 48, 1)
num_classes = 7

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation="relu", input_shape=img_size),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation="relu"),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dense(num_classes, activation="softmax")
])

model.compile(optimizer="adam",
              loss="categorical_crossentropy",
              metrics=["accuracy"])


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [20]:
import tensorflow as tf

img_size = (48, 48, 1)
num_classes = 7

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation=tf.nn.relu, input_shape=img_size),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation=tf.nn.relu),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(num_classes, activation=tf.nn.softmax)
])

model.compile(optimizer="adam",
              loss="categorical_crossentropy",
              metrics=["accuracy"])


In [23]:
import cv2
import numpy as np
import glob
import os

img_size = (48,48)

def load_affectnet_images(path_images, path_labels, limit=5000):
    X, y = [], []
    img_files = glob.glob(os.path.join(path_images, "*.jpg"))
    print(f"Found {len(img_files)} image files in {path_images}")

    for i, img_path in enumerate(img_files[:limit]):  # limit para no reventar RAM
        label_path = img_path.replace("images", "labels").replace(".jpg", ".txt")

        if not os.path.exists(label_path):
            print(f"Label file not found for {img_path}")
            continue

        # Leer imagen
        img = cv2.imread(img_path)
        if img is None:
            print(f"Could not read image file {img_path}")
            continue
        h, w, _ = img.shape

        # Leer etiquetas YOLO (clase, cx, cy, bw, bh)
        with open(label_path, "r") as f:
            for line in f:
                try:
                    cls, cx, cy, bw, bh = map(float, line.strip().split())
                    cls = int(cls)

                    # Convertir YOLO a pixeles
                    x1 = int((cx - bw/2) * w)
                    y1 = int((cy - bh/2) * h)
                    x2 = int((cx + bw/2) * w)
                    y2 = int((cy + bh/2) * h)

                    # Recortar rostro
                    face = img[y1:y2, x1:x2]
                    if face.size == 0:
                        print(f"Cropped face is empty for {img_path}")
                        continue

                    # Escala de grises y resize
                    face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
                    face = cv2.resize(face, img_size)
                    X.append(face)
                    y.append(cls)
                except Exception as e:
                    print(f"Error processing label file {label_path}: {e}")


    X = np.array(X).reshape(-1, 48, 48, 1) / 255.0
    y = np.array(y)
    return X, y

X, y = load_affectnet_images(
    "/content/dataset/affectnet/images",
    "/content/dataset/affectnet/labels",
    limit=5000
)
print("Dataset cargado:", X.shape, y.shape)

if X.shape[0] == 0:
    print("No data was loaded from AffectNet. Please check the paths and dataset structure.")

Found 0 image files in /content/dataset/affectnet/images
Dataset cargado: (0, 48, 48, 1) (0,)
No data was loaded from AffectNet. Please check the paths and dataset structure.


In [35]:
import glob, os

imgs = glob.glob("/content/dataset/AffectNet/YOLO_format/test/images/*.jpg")
labels = glob.glob("/content/dataset/AffectNet/YOLO_format/test/labels/*.txt")

print("Cantidad de imágenes:", len(imgs))
print("Cantidad de labels:", len(labels))


Cantidad de imágenes: 1752
Cantidad de labels: 2755


# ***este es otro ***

In [42]:
!pip install flask pyngrok


In [44]:
!ngrok config add-authtoken 31SvM5OXEuo6RytCNnvo6a5eoC5_7pzrSkHYayLugRygJyR1q



Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [48]:
from flask import Flask, request, render_template_string
from flask_ngrok import run_with_ngrok
from transformers import BeitImageProcessor, BeitForImageClassification
from PIL import Image
import torch

# Modelo
processor = BeitImageProcessor.from_pretrained("Tanneru/Facial-Emotion-Detection-BEIT-Large")
model = BeitForImageClassification.from_pretrained("Tanneru/Facial-Emotion-Detection-BEIT-Large")

app = Flask(__name__)
run_with_ngrok(app)  # 🔹 Aquí se encarga solo ngrok

HTML_PAGE = """
<!DOCTYPE html>
<html>
<head>
  <title>Detector de Emociones</title>
</head>
<body>
  <h2>Sube una imagen de rostro</h2>
  <form action="/predict" method="post" enctype="multipart/form-data">
    <input type="file" name="file">
    <input type="submit" value="Predecir">
  </form>
  {% if label %}
    <h3>Emoción predicha: {{ label }}</h3>
  {% endif %}
</body>
</html>
"""

@app.route("/", methods=["GET"])
def home():
    return render_template_string(HTML_PAGE)

@app.route("/predict", methods=["POST"])
def predict():
    if "file" not in request.files:
        return "No se subió ninguna imagen"
    file = request.files["file"]
    image = Image.open(file.stream).convert("RGB")

    inputs = processor(images=image, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    predicted_class_idx = outputs.logits.argmax(-1).item()
    label = model.config.id2label[predicted_class_idx]

    return render_template_string(HTML_PAGE, label=label)

# 🔹 Con flask_ngrok no pones port
app.run()


preprocessor_config.json:   0%|          | 0.00/228 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


config.json:   0%|          | 0.00/915 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/343M [00:00<?, ?B/s]

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
Exception in thread Thread-11:
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/urllib3/connection.py", line 198, in _new_conn
    sock = connection.create_connection(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/usr/local/lib/python3.12/dist-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/urllib3/connectionpool.py", line 787, in urlopen
    response = self._make_request(
           

In [49]:
!pip install pyngrok


In [50]:
from pyngrok import ngrok
from flask import Flask, request, render_template_string
import torch
from transformers import AutoImageProcessor, AutoModelForImageClassification
from PIL import Image
import torch.nn.functional as F

# Modelo HuggingFace
processor = AutoImageProcessor.from_pretrained("trpakov/vit-face-expression")
model = AutoModelForImageClassification.from_pretrained("trpakov/vit-face-expression")

# Flask app
app = Flask(__name__)

@app.route("/", methods=["GET", "POST"])
def index():
    html = """
    <h1>Detector de Emociones</h1>
    <form method="POST" enctype="multipart/form-data">
        <input type="file" name="image">
        <input type="submit" value="Detectar">
    </form>
    <div>{{ result|safe }}</div>
    """
    result = ""
    if request.method == "POST":
        file = request.files["image"]
        image = Image.open(file.stream).convert("RGB")

        inputs = processor(images=image, return_tensors="pt")
        with torch.no_grad():
            outputs = model(**inputs)
            probs = F.softmax(outputs.logits, dim=-1)[0]

        # Top 3 emociones
        top3 = torch.topk(probs, 3)
        result = "<h3>Resultados:</h3><ul>"
        for idx, score in zip(top3.indices, top3.values):
            label = model.config.id2label[idx.item()]
            result += f"<li>{label}: {score.item():.2f}</li>"
        result += "</ul>"

    return render_template_string(html, result=result)

# Crear túnel
public_url = ngrok.connect(5000)
print("🌍 Abre esta URL en tu navegador:", public_url)

app.run(port=5000)


🌍 Abre esta URL en tu navegador: NgrokTunnel: "https://c9561dc30046.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [20/Aug/2025 17:37:12] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Aug/2025 17:37:13] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [20/Aug/2025 17:37:20] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Aug/2025 17:37:36] "POST / HTTP/1.1" 200 -
